<a href="https://colab.research.google.com/github/itsvivan/Encryptix_tasks/blob/main/Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,f1_score,precision_score,recall_score,roc_auc_score,roc_curve
from sklearn.preprocessing import LabelEncoder

In [2]:
fd=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fraudTrain.csv')
fd.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [3]:
fd.shape

(1296675, 23)

In [4]:
fd.info()
fd.isna().sum()  # checking the null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [5]:
fd1=fd.drop(['Unnamed: 0','cc_num','trans_num','merch_lat','merch_long','lat','long','job','trans_date_trans_time','street','merchant','last','state'],axis=1)
fd1.shape # removing tht columns that are useful for the classification

(1296675, 10)

In [6]:
fd1.head()

,category,amt,first,gender,city,zip,city_pop,dob,unix_time,is_fraud
0,misc_net,4.97,Jennifer,F,Moravian Falls,28654,3495,1988-03-09,1325376018,0
1,grocery_pos,107.23,Stephanie,F,Orient,99160,149,1978-06-21,1325376044,0
2,entertainment,220.11,Edward,M,Malad City,83252,4154,1962-01-19,1325376051,0
3,gas_transport,45.00,Jeremy,M,Boulder,59632,1939,1967-01-12,1325376076,0
4,misc_pos,41.96,Tyler,M,Doe Hill,24433,99,1986-03-28,1325376186,0


In [7]:
fd.category.value_counts()
fd.city.value_counts()
fd.state.value_counts()



state
TX    94876
NY    83501
PA    79847
CA    56360
OH    46480
MI    46154
IL    43252
FL    42671
AL    40989
MO    38403
MN    31714
AR    31127
NC    30266
WI    29368
VA    29250
SC    29190
KY    28475
IN    27580
IA    26985
OK    26671
MD    26193
GA    26063
WV    25691
NJ    24603
NE    24168
KS    22996
MS    21188
LA    20965
WY    19322
WA    18924
OR    18597
TN    17554
ME    16505
NM    16407
ND    14786
CO    13880
MA    12376
SD    12324
VT    11768
MT    11754
AZ    10770
UT    10699
NH     8278
CT     7702
NV     5607
ID     5545
DC     3613
HI     2559
AK     2120
RI      550
DE        9
Name: count, dtype: int64

fd.category.value_counts()

In [8]:
columns_to_encode = ['category', 'gender','city','dob','unix_time','first']
label_encoder = LabelEncoder()
for column in columns_to_encode:
    fd1[column] = label_encoder.fit_transform(fd1[column])
fd1.head()

,category,amt,first,gender,city,zip,city_pop,dob,unix_time,is_fraud
0,8,4.97,162,0,526,28654,3495,779,0,0
1,4,107.23,309,0,612,99160,149,607,1,0
2,0,220.11,115,1,468,83252,4154,302,2,0
3,2,45.00,163,1,84,59632,1939,397,3,0
4,9,41.96,336,1,216,24433,99,734,4,0


In [9]:

corr_matrix=fd1.corr()
corr_matrix
# plt.figure(figsize=(10,10))
# sns.heatmap(corr_matrix,annot=True,cmap='coolwarm')

,category,amt,first,gender,city,zip,city_pop,dob,unix_time,is_fraud
category,1.000000,0.030867,-0.001931,-0.028259,0.000784,0.002371,0.009386,0.004347,0.000249,0.020205
amt,0.030867,1.000000,-0.003575,0.001034,-0.000565,0.001843,0.005818,0.010650,-0.000253,0.219404
first,-0.001931,-0.003575,1.000000,0.015197,-0.009211,0.004766,-0.066607,0.004203,0.000509,-0.003219
gender,-0.028259,0.001034,0.015197,1.000000,0.016039,-0.065951,-0.028649,-0.012185,-0.000946,0.007642
city,0.000784,-0.000565,-0.009211,0.016039,1.000000,0.074757,0.034923,0.014330,-0.000424,-0.002092
zip,0.002371,0.001843,0.004766,-0.065951,0.074757,1.000000,0.078467,-0.013300,0.000709,-0.002162
city_pop,0.009386,0.005818,-0.066607,-0.028649,0.034923,0.078467,1.000000,0.088756,-0.001679,0.002136
dob,0.004347,0.010650,0.004203,-0.012185,0.014330,-0.013300,0.088756,1.000000,0.003708,-0.012156
unix_time,0.000249,-0.000253,0.000509,-0.000946,-0.000424,0.000709,-0.001679,0.003708,1.000000,-0.004778
is_fraud,0.020205,0.219404,-0.003219,0.007642,-0.002092,-0.002162,0.002136,-0.012156,-0.004778,1.000000


In [12]:
y_train=fd1['is_fraud']
X_train=fd1.drop('is_fraud',axis=1)

In [13]:
y_train.value_counts()

is_fraud
0    1289169
1       7506
Name: count, dtype: int64

In [24]:
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fraudTest.csv')
test.head()


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [25]:
test.shape

(555719, 23)

In [27]:
test1=test.drop(['Unnamed: 0','cc_num','trans_num','merch_lat','merch_long','lat','long','job','trans_date_trans_time','street','merchant','last','state'],axis=1)
test1.shape

(555719, 10)

In [28]:
columns_to_encode = ['category', 'gender','city','dob','unix_time','first']
label_encoder = LabelEncoder()
for column in columns_to_encode:
    test1[column] = label_encoder.fit_transform(test1[column])
test1.head()

,category,amt,first,gender,city,zip,city_pop,dob,unix_time,is_fraud
0,10,2.86,151,1,157,29209,333497,376,0,0
1,10,29.84,163,0,16,84002,302,760,1,0
2,5,41.28,24,0,61,11710,34496,421,2,0
3,9,60.05,42,1,764,32780,54767,718,3,0
4,13,3.19,247,1,247,49632,1126,177,4,0


In [29]:
y_test=test1['is_fraud']
X_test=test1.drop('is_fraud',axis=1)

#**Trying Different Models**


In [11]:
models={
    'LogisticRegression':LogisticRegression(),
    'DecisionTreeClassifier':DecisionTreeClassifier(),
    'RandomForestClassifier':RandomForestClassifier()
}

In [32]:
k=list(models.values())
for i in range(len(list(models))):
  model=k[i]
  model.fit(X_train,y_train)

  y_train_pred=model.predict(X_train)
  y_test_pred=model.predict(X_test)

  #training performance
  model_train_accuracy=accuracy_score(y_train,y_train_pred)
  model_train_f1=f1_score(y_train,y_train_pred,average='weighted')
  model_train_precision=precision_score(y_train,y_train_pred)
  model_train_recall=recall_score(y_train,y_train_pred)
  model_train_rocauc=roc_auc_score(y_train,y_train_pred)

  #test performance
  model_test_accuracy=accuracy_score(y_test,y_test_pred)
  model_test_f1=f1_score(y_test,y_test_pred,average='weighted')
  model_test_precision=precision_score(y_test,y_test_pred)
  model_test_recall=recall_score(y_test,y_test_pred)
  model_test_rocauc=roc_auc_score(y_test,y_test_pred)

  print(list(models.keys())[i])
  print('Model performance for Training set')
  print("- Accuracy: {:.4f}".format(model_train_accuracy))
  print('- F1 score: {:.4f}'.format(model_train_f1))
  print('- Precision: {:.4f}'.format(model_train_precision))
  print('- Recall: {:.4f}'.format(model_train_recall))
  print('- ROC AUC: {:.4f}'.format(model_train_rocauc))

  print('----------------------------------')

  print('Model performance for Test set')
  print('- Accuracy: {:.4f}'.format(model_test_accuracy))
  print('- F1 score: {:.4f}'.format(model_test_f1))
  print('- Precision: {:.4f}'.format(model_test_precision))
  print('- Recall: {:.4f}'.format(model_test_recall))
  print('- ROC AUC: {:.4f}'.format(model_test_rocauc))



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
Model performance for Training set
- Accuracy: 0.9937
- F1 score: 0.9912
- Precision: 0.0800
- Recall: 0.0080
- ROC AUC: 0.5037
----------------------------------
Model performance for Test set
- Accuracy: 0.9954
- F1 score: 0.9940
- Precision: 0.0733
- Recall: 0.0159
- ROC AUC: 0.5075
DecisionTreeClassifier
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- ROC AUC: 1.0000
----------------------------------
Model performance for Test set
- Accuracy: 0.9963
- F1 score: 0.9964
- Precision: 0.5214
- Recall: 0.5907
- ROC AUC: 0.7943
RandomForestClassifier
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 0.9995
- ROC AUC: 0.9997
----------------------------------
Model performance for Test set
- Accuracy: 0.9978
- F1 score: 0.9977
- Precision: 0.7948
- Recall: 0.5958
- ROC AUC: 0.7976


On the basis of the test scores , we are using the Random forest classifier


#**Random Forest**

In [38]:
#Hyperparameter Training

# rf_params = {

#     'max_depth': [ 5,15,'None'],  # Maximum depth of individual trees
#     'max_features': [5,7,'auto'],  # Number of features to consider when looking for the best split
#     'min_samples_split': [2, 8, 15],  # Minimum samples required to split a node

#     'n_estimators': [200, 500,],  # Number of trees in the forest

# }


In [34]:
rf_params

{'max_depth': [5, 8, 10, 15, 'None'],
 'max_features': [5, 7, 'auto', 8],
 'min_samples_split': [2, 8, 15, 20],
 'n_estimators': [100, 200, 500, 1000]}

In [35]:
rf=RandomForestClassifier()

In [43]:
rf.fit(X_train,y_train)

RandomForestClassifier()

In [46]:
y_pred=rf.predict(X_test)

In [45]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.9978334374027161

In [42]:
# random_search = RandomizedSearchCV(estimator=rf,param_distributions=rf_params)


In [41]:
# random_search.fit(X_train, y_train)

In [ ]:
# best_params = random_search.best_params_
# best_model = random_search.best_estimator_

In [ ]:
# best_model.fit(X_train, y_train)

In [ ]:
y_train_pred=best.predict(X_train)

accuracy = accuracy_score(y, y_train1)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9999699230724738
